In [355]:
import boto3

In [356]:
dynamodb = boto3.resource('dynamodb')

In [357]:
# テーブル「Users」を作成する

dynamodb.create_table(
    TableName='Users', # テーブル名
    KeySchema=[
        {'AttributeName': 'UserId', 'KeyType': 'HASH'} # 'HASH': パーティションキー
    ],
    AttributeDefinitions=[
        {'AttributeName': 'UserId', 'AttributeType': 'S'} # 'S': 文字列型
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 10, # 1秒あたりの読み込み回数
        'WriteCapacityUnits': 10 # 1秒あたりの書き込み回数
    }
)

dynamodb.Table(name='Users')

In [360]:
# 作成直後は「CREATING」となる。
# このセルをCtrl+Enterで何度か実行し、「ACTIVE」になるまで待ってください。
dynamodb.Table(name='Users').table_status

'ACTIVE'

In [358]:
# テーブルを取得
table = dynamodb.Table(name='Users')

In [361]:
table

dynamodb.Table(name='Users')

In [362]:
# put_itemで、項目を作成する

table.put_item(
    Item={
        'UserId': 'Yamada',
        'Password': 'pass1'
    }
)

{'ResponseMetadata': {'RequestId': 'N7VR0910TFCJUUM8HT9SJ78593VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 19 Dec 2018 22:01:53 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'N7VR0910TFCJUUM8HT9SJ78593VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [363]:
# get_itemで、項目を取得する

table.get_item(
    Key={'UserId': 'Yamada'}
)

{'Item': {'Password': 'pass1', 'UserId': 'Yamada'},
 'ResponseMetadata': {'RequestId': 'O03H9OJ08QOOUTTM7T3AC8DFJNVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 19 Dec 2018 22:01:56 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'O03H9OJ08QOOUTTM7T3AC8DFJNVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3004546007'},
  'RetryAttempts': 0}}

In [364]:
# get_itemは、Item（項目そのものの情報）と
# ResponseMetadata（APIのレスポンス情報）を返す。
# Itemだけ取得するには以下のようにすればよい

table.get_item(
    Key={'UserId': 'Yamada'}
)['Item']

{'Password': 'pass1', 'UserId': 'Yamada'}

In [365]:
# もう一度同じキー（パーティションキー）でput_itemすると、
# 項目全体が、新しい項目で上書きされる。

table.put_item(
    Item={
        'UserId': 'Yamada',
        'Name': 'Yamada Taro'
    }
)

{'ResponseMetadata': {'RequestId': 'BCH76H7A11DC5PJ0HH8E909A6FVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 19 Dec 2018 22:02:01 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'BCH76H7A11DC5PJ0HH8E909A6FVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [366]:
# get_itemで取り出して確認。
# 項目全体が、新しい項目で上書きされたため、Passwordがなくなっているところに注目。

table.get_item(
    Key={'UserId': 'Yamada'}
)['Item']

{'UserId': 'Yamada', 'Name': 'Yamada Taro'}

In [367]:
# 既存の項目を残したまま、項目に新しい属性をセットする（追加する）には、
# put_itemではなくupdate_itemを使う。

table.update_item(
    Key={'UserId': 'Yamada'},
    UpdateExpression='SET Password = :Password', # 更新式
    ExpressionAttributeValues={ # 式の属性値
        ':Password': 'pass1'
    }
)

# get_itemで取り出して確認
table.get_item(
    Key={'UserId': 'Yamada'}
)['Item']

{'Password': 'pass1', 'UserId': 'Yamada', 'Name': 'Yamada Taro'}

In [368]:
# なお実際のシステムで、パスワードをDB保存する際は
# 生パスワードではなく、SHA256ハッシュ値などを計算してそれを保存すべき。

import hashlib
password = 'pass1'
hashlib.sha256(password.encode()).hexdigest()

'e6c3da5b206634d7f3f3586d747ffdb36b5c675757b380c6a5fe5c570c714349'

In [369]:
# Age（年齢）を追加してみる

table.update_item(
    Key={'UserId': 'Yamada'},
    UpdateExpression='SET Age = :Age',
    ExpressionAttributeValues={
        ':Age': 22
    }
)

# get_itemで取り出して確認. 
# Ageは 22ではなく Decimal('22') として出てくる。

table.get_item(
    Key={'UserId': 'Yamada'}
)['Item']

{'UserId': 'Yamada',
 'Password': 'pass1',
 'Age': Decimal('22'),
 'Name': 'Yamada Taro'}

In [370]:
# Decimalは、Pythonのクラスの一種。decimalモジュールで定義されている。
# Decimalは「十進浮動小数点」であり、通常の浮動小数点数よりも正確に数を扱えるのが特徴。
# Decimalについて詳しくは以下を参照。
# https://docs.python.jp/3/library/decimal.html

from decimal import *
Decimal('22')

Decimal('22')

In [371]:
# Decimalを普通の整数にするにはint()を使う
int(Decimal('22'))

22

In [372]:
# 小数点以下は切り捨てることができる
int(Decimal('22.49'))

22

In [373]:
# Decimalを「浮動小数点数」(float型)にするにはfloat()を使う
float(Decimal('22.22'))

22.22

In [374]:
# 特に、浮動小数点数（小数点付きの数）をDynamoDBに指定する場合は
# 明示的にDecimal()を使わなければいけない点に注意。

table.update_item(
    Key={'UserId': 'Yamada'},
    UpdateExpression='SET Height = :Height',
    ExpressionAttributeValues={
        ':Height': Decimal('165.5') # 165.5ではなくDecimal('165.5')とする必要がある
    }
)

# get_itemで取り出して確認. 

table.get_item(
    Key={'UserId': 'Yamada'}
)['Item']

{'UserId': 'Yamada',
 'Height': Decimal('165.5'),
 'Password': 'pass1',
 'Age': Decimal('22'),
 'Name': 'Yamada Taro'}

In [375]:
# get_itemで、ProjectionExpressionを使用すると、
# 指定した属性だけを取り出すことができる
# これを「射影」という。

table.get_item(
    Key={'UserId': 'Yamada'},
    ProjectionExpression='Height'
)['Item']

{'Height': Decimal('165.5')}

In [376]:
# 「射影」 (ProjectionExpression) では,
# カンマ区切りで複数の属性を指定することができる

table.get_item(
    Key={'UserId': 'Yamada'},
    ProjectionExpression='Height,Age'
)['Item']

{'Age': Decimal('22'), 'Height': Decimal('165.5')}

In [377]:
# Height（身長）を削除

table.update_item(
    Key={'UserId': 'Yamada'},
    UpdateExpression='REMOVE Height',
)

# get_itemで取り出して確認. 

table.get_item(
    Key={'UserId': 'Yamada'}
)['Item']

{'UserId': 'Yamada',
 'Password': 'pass1',
 'Age': Decimal('22'),
 'Name': 'Yamada Taro'}

In [ ]:
# Age（年齢）を1増やす

table.update_item(
    Key={'UserId': 'Yamada'},
    UpdateExpression='ADD Age :Age',
    ExpressionAttributeValues={
        ':Age': 1
    }
)

# get_itemで取り出して確認. 

table.get_item(
    Key={'UserId': 'Yamada'}
)['Item']

In [ ]:
# Age（年齢）を1減らす(!?)

table.update_item(
    Key={'UserId': 'Yamada'},
    UpdateExpression='ADD Age :Age',
    ExpressionAttributeValues={
        ':Age': -1 # 数を減らすには負の数をADDすればよい
    }
)

# get_itemで取り出して確認. 

table.get_item(
    Key={'UserId': 'Yamada'}
)['Item']

In [ ]:
# Age（年齢）を1減らす(!?)

table.update_item(
    Key={'UserId': 'Yamada'},
    UpdateExpression='ADD FavoritePets :Pets',
    ExpressionAttributeValues={
        ':Pets': set(['Dog'])
    }
)

# get_itemで取り出して確認. 

table.get_item(
    Key={'UserId': 'Yamada'}
)['Item']

In [ ]:
# 「好きなペットの種類」リストを作成し、Dogを入れる。

table.update_item(
    Key={'UserId': 'Yamada'},
    UpdateExpression='SET FavoritePets = :Pets',
    ExpressionAttributeValues={
        ':Pets': ['dog']
    }
)

# get_itemで取り出して確認. 

table.get_item(
    Key={'UserId': 'Yamada'}
)['Item']

In [ ]:
# 「好きなペットの種類」の末尾にCatを追加

table.update_item(
    Key={'UserId': 'Yamada'},
    UpdateExpression='SET FavoritePets = list_append(FavoritePets, :Pets)',
    ExpressionAttributeValues={
        ':Pets': ['cat']
    }
)

# get_itemで取り出して確認. 

table.get_item(
    Key={'UserId': 'Yamada'}
)['Item']

In [ ]:
# 「好きなペットの種類」の先頭にハムスターを追加

table.update_item(
    Key={'UserId': 'Yamada'},
    UpdateExpression='SET FavoritePets = list_append(:Pets, FavoritePets)',
    ExpressionAttributeValues={
        ':Pets': ['hamster']
    }
)

# get_itemで取り出して確認. 

table.get_item(
    Key={'UserId': 'Yamada'}
)['Item']

In [ ]:
# 「好きなペットの種類」の1番目（Dog）をRabbitに変更
# 配列は0からカウントする

table.update_item(
    Key={'UserId': 'Yamada'},
    UpdateExpression='SET FavoritePets[1] = :Pet',
    ExpressionAttributeValues={
        ':Pet': 'rabbit'
    }
)

# get_itemで取り出して確認. 

table.get_item(
    Key={'UserId': 'Yamada'}
)['Item']

In [ ]:
# 「好きなペットの種類」の1番目（Rabbit）を削除
# 配列は0からカウントする

table.update_item(
    Key={'UserId': 'Yamada'},
    UpdateExpression='REMOVE FavoritePets[1]',
)

# get_itemで取り出して確認. 

table.get_item(
    Key={'UserId': 'Yamada'}
)['Item']

In [ ]:
# まずPythonでのset（集合）の使い方を確認。
# 文字列を指定すると、文字単位に分割される。
# かつ、「o」はまとめられて1つになる（重複不可）
set('facebook')

In [ ]:
# 文字単位に分解されないようにするためには、引数をリスト [ ... ] にする
set(['facebook'])

In [ ]:
# たくさんの要素を一度に指定できる
set(['facebook', 'twitter', 'instagram', 'snow', 'tiktok'])

In [ ]:
# 「使用中のアプリ」を「セット」として記録する
# セット（集合）は、リスト（配列）とは異なり、順番を持たない。要素は重複できない。

table.update_item(
    Key={'UserId': 'Yamada'},
    UpdateExpression='SET Apps = :Apps',
    ExpressionAttributeValues={
        ':Apps': set(['facebook'])
    }
)

# get_itemで取り出して確認. 

table.get_item(
    Key={'UserId': 'Yamada'}
)['Item']

In [ ]:
# セットに要素を追加する

table.update_item(
    Key={'UserId': 'Yamada'},
    UpdateExpression='ADD Apps :Apps',
    ExpressionAttributeValues={
        ':Apps': set(['twitter', 'tiktok', 'snow', 'instagram'])
    }
)

table.get_item(
    Key={'UserId': 'Yamada'}
)['Item']


In [ ]:
# セットから要素を削除するにはDELETEを使う

table.update_item(
    Key={'UserId': 'Yamada'},
    UpdateExpression='DELETE Apps :Apps',
    ExpressionAttributeValues={
        ':Apps': set(['facebook', 'twitter'])
    }
)

table.get_item(
    Key={'UserId': 'Yamada'}
)['Item']


In [ ]:
# 住所については「郵便番号」,
# 「住所1(都道府県)」,「住所2(市区町村)」,「住所3(丁番・建物名・部屋番号)」
# のように複数の項目に分かれるため、
# マップ（ネストしたマップ）として記録する。

# まずAddressに {} （空のマップ）を登録。

table.update_item(
    Key={'UserId': 'Yamada'},
    UpdateExpression='SET Address = :Address',
    ExpressionAttributeValues={
        ':Address': {}
    }
)

table.get_item(
    Key={'UserId': 'Yamada'}
)['Item']

In [ ]:
# 住所の郵便番号 (Address.PostalCode)をセット

table.update_item(
    Key={'UserId': 'Yamada'},
    UpdateExpression='SET Address.PostalCode = :PostalCode',
    ExpressionAttributeValues={
        ':PostalCode': '104-0061'
    }
)

table.get_item(
    Key={'UserId': 'Yamada'}
)['Item']


In [ ]:
# 住所1(Address.Address1), 住所2, 住所3を一度にセット
table.update_item(
    Key={'UserId': 'Yamada'},
    UpdateExpression='''
        SET Address.Address1 = :Address1,
            Address.Address2 = :Address2,
            Address.Address3 = :Address3
    ''',
    ExpressionAttributeValues={
        ':Address1': '東京都',
        ':Address2': '中央区銀座',
        ':Address3': '1丁目1番1号 111号室',
    }
)

table.get_item(
    Key={'UserId': 'Yamada'}
)['Item']


In [ ]:
# 項目の削除
table.delete_item(Key={'UserId': 'Yamada'})

In [ ]:
# テーブルの削除
table.delete()

In [ ]:
# 削除直後は「DELETING」（削除中）となる。
# 削除が完了すると、テーブルの情報取得処理自体が失敗してResourceNotFoundExceptionとなる。
# このセルをCtrl+Enterで何度か実行し、「すでにテーブルは削除されています」と表示されるまで待ってください。
try:
    status = dynamodb.Table(name='Users').table_status
    print(status)
except dynamodb.meta.client.exceptions.ResourceNotFoundException as err:
    print('すでにテーブルは削除されています')

In [ ]:
# まとめ

# dynamodb.create_table(...) テーブルの作成
# table =  dynamodb.Table(...) テーブルオブジェクトの取得
# table.put_item(...) テーブルに項目を追加/上書き
# table.update_item(...) 項目の属性を追加/更新/削除
# table.delete_item(...) 項目を削除
# table.delete() テーブルの削除

# 属性の値として、文字列、数値、リスト、セット、マップなどが使用できる。

# 数値は増減できる
# リストには順序がある
# セットには順序がなく、要素の重複が許可されない
# マップ（ネストしたマップ）は、1つの属性値の中に、複数のキー・値を持つ